In [184]:
import pandapower as pp
import pandapower.plotting as pplot
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

net = pp.create_empty_network()
p = 150
q = 100

# Crear un nuevo tipo de línea
b1 = pp.create_bus(net, vn_kv=110.)
b2 = pp.create_bus(net, vn_kv=220.)
b3 = pp.create_bus(net, vn_kv=220.)
b4 = pp.create_bus(net, vn_kv=220.)
b5 = pp.create_bus(net, vn_kv=220.)
b6 = pp.create_bus(net, vn_kv=220.)
b7 = pp.create_bus(net, vn_kv=220.)
trafo = pp.create_transformer(net, b2, b1, std_type="100 MVA 220/110 kV", name="trafo1")
# Crear una línea utilizando el nuevo tipo de línea
linea1 = pp.create_line(net, from_bus=b2, to_bus=b3, length_km=10., std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="linea1")
linea2 = pp.create_line(net, from_bus=b3, to_bus=b4, length_km=15., std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="linea2")
linea3 = pp.create_line(net, from_bus=b4, to_bus=b5, length_km=20., std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="linea3")
linea4 = pp.create_line(net, from_bus=b5, to_bus=b6, length_km=15., std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="linea4")
linea5 = pp.create_line(net, from_bus=b6, to_bus=b7, length_km=30., std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="linea5")
linea6 = pp.create_line(net, from_bus=b7, to_bus=b2, length_km=10., std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="linea6")
pp.create_ext_grid(net, bus=b1, vm_pu=1., name="barra_ext")
pp.create_load(net, bus=b3, p_mw=0.2*p, q_mvar=0.2*q, name="carga1")
pp.create_load(net, bus=b4, p_mw=0.35*p, q_mvar=0.35*q, name="carga2")
pp.create_load(net, bus=b5, p_mw=0.15*p, q_mvar=0.15*q, name="carga3")
pp.create_load(net, bus=b6, p_mw=0.6*p, q_mvar=0.6*q, name="carga4")
pp.create_load(net, bus=b7, p_mw=0.1*p, q_mvar=0.1*q, name="carga5")
pp.runpp(net)

In [185]:
#Potencias inyectadas en las barras PQ (MW y MVAR)
P_inyectado = np.array([150, 30, 52.5, 22.5, 90, 15])  # MW
Q_inyectado = np.array([100, 20, 35, 15, 60, 10])  # MVAR

#Voltajes de las barras PQ (inicialización, por ser valores ficticios para el método)
V = np.array([110, 220, 220, 220, 220, 220, 220])
theta = np.radians(np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))  # Inicialmente supuesto que todos los voltajes son 1 pu

#Construcción de la matriz Ybus (parte real e imaginaria)
Ybus = net._ppc['internal']['Ybus']
np.set_printoptions(precision=3, suppress=True)
Ybus_array = Ybus.toarray()
Ybus_df = pd.DataFrame(Ybus_array)
pd.set_option('display.max_columns', None)  # Asegura mostrar todas las columnas
pd.set_option('display.width', 1000)  # Ajusta el ancho para la visualización
pd.set_option('display.float_format', '{:.3f}'.format)  # Formato de los números flotantes
# Convertir Ybus a una matriz densa para visualización
print("Matriz de Admitancia de Barra (Ybus):")
Y_real = np.real(Ybus)
Y_imag = np.imag(Ybus)
#Método de Newton-Raphson para encontrar voltajes y ángulos
def calcular_derivadas(V, theta, Y_real, Y_imag):
    n = len(V)
    H = np.zeros((n, n))
    N = np.zeros((n, n))
    M = np.zeros((n, n))
    L = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                H[i, j] = -V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
                N[i, j] = -V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                M[i, j] = V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                L[i, j] = -V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
            else:
                H[i, i] = V[i]**2 * Y_imag[i, i] + np.sum(V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j])))
                N[i, i] = -V[i] * Y_real[i, i] - (np.sum(V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))) / V[i])
                M[i, i] = V[i]**2 * Y_real[i, i] - np.sum(V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j])))
                L[i, i] = V[i] * Y_imag[i, i] - (np.sum(V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))) / V[i])

    return H, N, M, L

def calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado):
    n = len(V)
    delta_P = np.zeros(n)
    delta_Q = np.zeros(n)

    for i in range(n):
        sum_P = 0.0
        sum_Q = 0.0
        for j in range(n):
            sum_P += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
            sum_Q += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))

        delta_P[i] = P_inyectado[i] - sum_P
        delta_Q[i] = Q_inyectado[i] - sum_Q

    f = np.concatenate((delta_P, delta_Q))
    return f

    
def newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado, tol=1e-6, max_iter=500):
    iter = 0
    error = tol + 1

    while error > tol and iter < max_iter:
        H, N, M, L = calcular_derivadas(V, theta, Y_real, Y_imag)
        f = calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado)

        J1 = np.block([
            [H, N],
            [M, L]
        ])

        incremento = np.linalg.solve(J1, -f)
        delta_theta = incremento[:len(theta)]
        delta_V = incremento[len(theta):]

        theta += delta_theta
        V += delta_V

        error = np.max(np.abs(f))
        iter += 1

    if error > tol:
        print(f'El método de Newton-Raphson no convergió después de {max_iter} iteraciones.')

    return V, np.degrees(theta)
#Suponiendo voltajes y ángulos iniciales (ficticios para iniciar el método)
V_ini = np.ones(6)
theta_ini = np.zeros(6)

#Aplicar el método de Newton-Raphson
V_resultado, theta_resultado = newton_raphson(V_ini, np.radians(theta_ini), Y_real, Y_imag, P_inyectado, Q_inyectado)

#Mostrar resultados
print("Resultados del método de Newton-Raphson:")
print("Voltajes (en p.u.):", V_resultado)
print("Ángulos (en grados):", theta_resultado)

Matriz de Admitancia de Barra (Ybus):
El método de Newton-Raphson no convergió después de 500 iteraciones.
Resultados del método de Newton-Raphson:
Voltajes (en p.u.): [4.913 0.019 0.032 0.021 0.053 0.014]
Ángulos (en grados): [-40753.551   -351.763   -552.277   -428.88    -305.938   -486.323]
